In [1]:
#-*- coding:utf-8 -*-
#'''
# Created on 18-10-15
#
# @Author: Greg Gao(laygin)
#'''
import os
import numpy as np
import cv2
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import imutils
from pixellink_model import create_pixellink_model
from pixellink_utils import *

Using TensorFlow backend.


In [2]:
r_mean = 123.
g_mean = 117.
b_mean = 104.
rgb_mean = [r_mean, g_mean, b_mean]


img_path = './samples/TOP_29_121222.jpg'
save_weights = './weights/pixellink.h5'
model = create_pixellink_model(acf='relu')
model.load_weights(save_weights)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
image = cv2.imread(img_path)
image, *r = resize_image(image)
image_ori = image.copy()
image = image[...,::-1] - rgb_mean
image = np.expand_dims(image, axis=0)

pixel_pos_scores, link_pos_scores = model.predict(image)
pixel_pos_scores = softmax(pixel_pos_scores, axis=-1)
link_pos_scores_reshaped = link_pos_scores.reshape(link_pos_scores.shape[:-1]+(8, 2))
link_pos_scores = softmax(link_pos_scores_reshaped, axis=-1)

#masks = decode_batch(pixel_pos_scores, link_pos_scores, pixel_conf_threshold=0.75, link_conf_threshold=0.9)
masks = decode_batch(pixel_pos_scores, link_pos_scores, pixel_conf_threshold=0.9, link_conf_threshold=0.95)

bboxes = mask_to_bboxes(masks[0], image_ori.shape, min_area=500, min_height=20)


In [4]:
len(bboxes)

3

In [107]:
image_c = image_ori.copy()
#cv2.line(image_c,(308,352),(691,439),(0,0,255),2)

num_run = 0
BREAK_POINT = 1
for box in bboxes:
    
    num_run += 1
    
    # 裁切區域的 x 與 y 座標（左上角）
    x = box[2]
    y = box[5]
    
    # 裁切區域的長度與寬度
    h = box[1] - box[5]
    w = box[6] - box[2]
    

    # 裁切圖片
    crop_img = image_c[y:y+h, x:x+w]

    if num_run == BREAK_POINT:
        break

[308 439 308 352 691 352 691 439]
(308, 352)


In [14]:
image_c = image_ori.copy()
#cv2.line(image_c,(308,352),(691,439),(0,0,255),2)

num_run = 0
BREAK_POINT = 3
for box in bboxes:
    points = np.reshape(box, [4, 2])
    
    pts1 = tuple(points[1])
    print(pts1)
    
    pts3 = tuple(points[3])
    print(pts3)
    
    #break
    #cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
    cv2.circle(image_c, pts1, 5, (25,255,255))  #26
    cv2.circle(image_c, pts3, 5, (25,255,25))
    
    
    
    #cv2.line(image_c,tuple(points[0]),tuple(points[1]),(0,0,255),2)
    #cv2.line(image_c,tuple(points[0]),tuple(points[3]),(0,0,255),2)
    #cv2.line(image_c,tuple(points[1]),tuple(points[2]),(0,0,255),2)
    #cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
    
    if num_run == BREAK_POINT:
        break

(308, 352)
(691, 439)
(303, 520)
(699, 440)
(392, 520)
(479, 595)


In [24]:
image_c = image_ori.copy()
#cv2.line(image_c,(308,352),(691,439),(0,0,255),2)

num_run = 0
BREAK_POINT = 3
for box in bboxes:
    points = np.reshape(box, [4, 2])
    print(points)

    
    #break
    #cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
    
    cv2.line(image_c,tuple(points[0]),tuple(points[1]),(0,0,255),2)
    cv2.line(image_c,tuple(points[0]),tuple(points[3]),(0,0,255),2)
    cv2.line(image_c,tuple(points[1]),tuple(points[2]),(0,0,255),2)
    cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
    
    if num_run == BREAK_POINT:
        break

[[308 439]
 [308 352]
 [691 352]
 [691 439]]
[[698 525]
 [303 520]
 [304 435]
 [699 440]]
[[392 595]
 [392 520]
 [479 520]
 [479 595]]


In [108]:
cv2.imshow('image', image_c)
cv2.imshow('crop_img', crop_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite('TOP_29_121222_res.jpg', image_c)
#cv2.imwrite('TOP_29_121222_res_crop_img03.jpg', crop_img)